In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

directory="/home/ubuntu/working/spark-examples/data"
trip_files="trips/*"

trips_df = spark.read.csv(f"file://{directory}/{trip_files}", inferSchema=True, header=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/03 10:53:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
trips_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [3]:
trips_df.createOrReplaceTempView("trips")

# 단일 회귀
- 독립 변수 X가 하나, 종속 변수 y도 하나인 회귀 방식
- $\hat{y} = wx+b$
- 운행 거리(`trips_distance`  $x$)에 따른 요금(`total_amount` - $\hat{y}$)를 예측

In [4]:
# 데이터 마트 구성
query = """
SELECT
    trip_distance,
    total_amount
FROM 
    trips
WHERE total_amount < 5000
  AND total_amount > 0
  AND passenger_count < 5
  AND trip_distance > 0
  AND trip_distance < 500
  AND TO_DATE(tpep_pickup_datetime) >= '2021-01-01'
  AND TO_DATE(tpep_pickup_datetime) <  '2021-08-01'
"""

data_df = spark.sql(query)
data_df.createOrReplaceTempView("data") # 데이터 마트 생성

In [5]:
data_df.show()

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         16.5|       70.07|
|         1.13|       11.16|
|         2.68|       18.59|
|         12.4|        43.8|
|          9.7|        32.3|
|          9.3|       43.67|
|         9.58|        46.1|
|         16.2|        45.3|
|         3.58|        19.3|
|         0.91|        14.8|
|         2.57|        12.8|
|          0.4|         5.3|
|         3.26|        17.3|
|        13.41|       47.25|
|         18.3|       61.42|
|         1.53|       14.16|
|          2.0|        11.8|
|         16.6|       54.96|
|         15.5|       56.25|
|          1.3|        16.8|
+-------------+------------+
only showing top 20 rows



In [6]:
# 훈련 / 테스트 데이터 나누기
train_df, test_df = data_df.randomSplit([0.8, 0.2], seed=42)

In [7]:
# 훈련 데이터는 캐싱
train_df.cache()

DataFrame[trip_distance: double, total_amount: double]

In [8]:
train_df.count(), test_df.count()

(10660955, 2665176)

`feature`가 하나 밖에 없지만 `VectorAssembler`를 활용해서 1차원 배열 형식으로 각각의 데이터를 만들어 준다.

In [9]:
from pyspark.ml.feature import VectorAssembler

vec_assembler = VectorAssembler(inputCols=["trip_distance"], outputCol="features")
vec_train_df = vec_assembler.transform(train_df)

vec_train_df.show(5)

+-------------+------------+--------+
|trip_distance|total_amount|features|
+-------------+------------+--------+
|         0.01|        3.05|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
|         0.01|         3.3|  [0.01]|
+-------------+------------+--------+
only showing top 5 rows



In [10]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(
    maxIter=50,
    labelCol='total_amount',
    featuresCol='features'
)

lr_model = lr.fit(vec_train_df)

23/08/03 11:13:25 WARN Instrumentation: [fd63053e] regParam is zero, which might cause numerical instability and overfitting.
23/08/03 11:13:26 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/08/03 11:13:37 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [11]:
type(lr_model)

pyspark.ml.regression.LinearRegressionModel

In [12]:
vec_test_df = vec_assembler.transform(test_df)
predictions = lr_model.transform(vec_test_df) # 예측 할 때도 transform
predictions.show(5)

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
|         0.01|         3.3|  [0.01]|9.418361874707077|
+-------------+------------+--------+-----------------+
only showing top 5 rows



In [13]:
# Evaluator를 쓰지 않고 Train 데이터에 대한 RMSE 확인
lr_model.summary.rootMeanSquaredError

6.308435272654453

In [14]:
lr_model.summary.r2

0.765849929974943

In [15]:
spark.stop()